In [2]:
import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import matplotlib.pyplot as plt            # library for visualization
import random 
from sklearn.model_selection import train_test_split
import numpy as np 
import pandas as pd 
import re
%matplotlib inline
import pandas as pd

In [3]:
df=pd.read_csv("reviews.tsv", sep='\t')

In [4]:
#features = df.iloc[:, 10].values
labels = df.iloc[:, 1].values

In [5]:
print(labels)

[4. 1. 4. ... 5. 1. 1.]


In [6]:
labels = df['rating']

In [7]:
labels

0         4.0
1         1.0
2         4.0
3         1.0
4         5.0
         ... 
328640    5.0
328641    5.0
328642    5.0
328643    1.0
328644    1.0
Name: rating, Length: 328645, dtype: float64

In [8]:
reviews = df.iloc[:, 2].values

In [9]:
print(reviews)

['Вкусная питца, естт летнее кафе, доставка, приемлемые цены!'
 'Думал можно днём с детьми посидеть на открытой площадке, но больше туда не придем. Курят все кому не лень. А приходить с мыслью, ну а вдруг курить не будут? - Глупая лотерея.  Далеко нам ещё до Европы...'
 'Есть веранда, вкусная пицца, делают по половине, паковка сзади заведения всегда занята, туалеты грязные'
 ... 'Все понравилось! Пицца норм'
 'обслуживание нормальное, соотношение цена/качество порции не соответствует вообще, в карбонате попался волос, ужасное место '
 'Ооочень долгое обслуживание.  Необоснованно высокий ценник. К тому же владелец ужасный хам. К немногочисленным посетителям и то относится неуважительно. Сидя за соседним столом, назвал клиентов нищебродами, так чтобы всем вокруг было слышно. Место не рекомендуем. ']


In [10]:
reviews = df['review_text']

In [11]:
reviews

0         Вкусная питца, естт летнее кафе, доставка, при...
1         Думал можно днём с детьми посидеть на открытой...
2         Есть веранда, вкусная пицца, делают по половин...
3                          Уже второй раз забивают на заказ
4                                       Любимая пиццерия! 👍
                                ...                        
328640                              Очень вкусная пицца!!!!
328641    Лучший борщ на Роза Хутор!!! Большая порция, т...
328642                          Все понравилось! Пицца норм
328643    обслуживание нормальное, соотношение цена/каче...
328644    Ооочень долгое обслуживание.  Необоснованно вы...
Name: review_text, Length: 328645, dtype: object

In [12]:
cleaned = df.loc[df['rating'].isin([1, 2, 3, 4, 5])]
print(cleaned)

           permalink  rating  \
0         1000095379     4.0   
1         1000095379     1.0   
2         1000095379     4.0   
3         1000095379     1.0   
4         1000095379     5.0   
...              ...     ...   
328640  245591834558     5.0   
328641  245591834558     5.0   
328642  245591834558     5.0   
328643  245591834558     1.0   
328644  245591834558     1.0   

                                              review_text  
0       Вкусная питца, естт летнее кафе, доставка, при...  
1       Думал можно днём с детьми посидеть на открытой...  
2       Есть веранда, вкусная пицца, делают по половин...  
3                        Уже второй раз забивают на заказ  
4                                     Любимая пиццерия! 👍  
...                                                   ...  
328640                            Очень вкусная пицца!!!!  
328641  Лучший борщ на Роза Хутор!!! Большая порция, т...  
328642                        Все понравилось! Пицца норм  
328643  обслужи

In [13]:
print(cleaned['rating'])

0         4.0
1         1.0
2         4.0
3         1.0
4         5.0
         ... 
328640    5.0
328641    5.0
328642    5.0
328643    1.0
328644    1.0
Name: rating, Length: 328369, dtype: float64


In [19]:
X, y = np.arange(10).reshape((5, 2)), (1, 2, 3, 4, 5)#range(5)
# how to put our data in it?

In [20]:
print(X)
print(y)

[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
(1, 2, 3, 4, 5)


In [40]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42)

In [41]:
train_test_split(y, shuffle=False)

[[0, 1, 2], [3, 4]]